# LexRankサンプル

In [19]:
import math
import numpy

In [20]:
def lex_rank(sentences, n, t = 0.1):
    """
    LexRankで文章を要約する．
    @param  sentences: list
        文章([[w1,w2,w3],[w1,w3,w4,w5],..]のような文リスト)
    @param  n: int
        文章に含まれる文の数
    @param  t: float
        コサイン類似度の閾値(default 0.1)
    @return : list
        LexRank
    """
    cosine_matrix = numpy.zeros((n, n))
    degrees = numpy.zeros((n,))
    l = []

     # 1. 隣接行列の作成
    for i in range(n):
        for j in range(n):
            cosine_matrix[i][j] = idf_modified_cosine(sentences, sentences[i], sentences[j])
            if cosine_matrix[i][j] > t:
                cosine_matrix[i][j] = 1
                degrees[i] += 1
            else:
                cosine_matrix[i][j] = 0

    # 2.LexRank計算
    for i in range(n):
        for j in range(n):
            cosine_matrix[i][j] = cosine_matrix[i][j] / degrees[i]

    ratings = power_method(cosine_matrix, n, t)

    return zip(sentences, ratings)


In [21]:
def idf_modified_cosine(sentences, sentence1, sentence2):
    """
    2文間のコサイン類似度を計算
    @param  sentence1: list
        文1([w1,w2,w3]のような単語リスト)
    @param  sentence2: list
        文2([w1,w2,w3]のような単語リスト)
    @param  sentences: list
        文章([[w1,w2,w3],[w1,w3,w4,w5],..]のような単語リスト)
    @return : float
        コサイン類似度
    """
    tf1 = compute_tf(sentence1)
    tf2 = compute_tf(sentence2)
    idf_metrics = compute_idf(sentences)
    return cosine_similarity(sentence1, sentence2, tf1, tf2, idf_metrics)

In [22]:
from collections import Counter


def compute_tf(sentence):
    """
    TFを計算
    @param  sentence: list
        文([w1,w2,w3]のような単語リスト)
    @return : list
        TFリスト
    """
    tf_values = Counter(sentence)
    tf_metrics = {}

    max_tf = find_tf_max(tf_values)

    for term, tf in tf_values.items():
        tf_metrics[term] = tf / max_tf

    return tf_metrics


def find_tf_max(terms):
    """
    単語の最大出現頻度を探索
    @param  terms: list
        単語の出現頻度
    @return : int
        単語の最大出現頻度
    """
    return max(terms.values()) if terms else 1


def compute_idf(sentences):
    """
    文章中の単語のIDF値を計算
    @param sentences: list
        文章([[w1,w2,w3],[w1,w3,w4,w5],..]のような単語リスト)
    @return: list
        IDFリスト
    """
    idf_metrics = {}
    sentences_count = len(sentences)

    for sentence in sentences:
        for term in sentence:
            if term not in idf_metrics:
                n_j = sum(1 for s in sentences if term in s)
                idf_metrics[term] = math.log(sentences_count / (1 + n_j))

    return idf_metrics


def cosine_similarity(sentence1, sentence2, tf1, tf2, idf_metrics):
    """
    コサイン類似度を計算
    @param  sentence1: list
        文1([w1,w2,w3]のような単語リスト)
    @param  sentence2: list
        文2([w1,w2,w3]のような単語リスト)
    @param  tf1: list
        文1のTFリスト
    @param  tf2: list
        文2のTFリスト
    @param  idf_metrics: list
        文章のIDFリスト
    @return : float
        コサイン類似度
    """
    unique_words1 = set(sentence1)
    unique_words2 = set(sentence2)
    common_words = unique_words1 & unique_words2

    numerator = sum((tf1[t] * tf2[t] * idf_metrics[t] ** 2) for t in common_words)
    denominator1 = sum((tf1[t] * idf_metrics[t]) ** 2 for t in unique_words1)
    denominator2 = sum((tf2[t] * idf_metrics[t]) ** 2 for t in unique_words2)

    if denominator1 > 0 and denominator2 > 0:
        return numerator / (math.sqrt(denominator1) * math.sqrt(denominator2))
    else:
        return 0.0    

In [23]:
def power_method(cosine_matrix, n, e):
    """
    べき乗法を行なう
    @param  scosine_matrix: list
        確率行列
    @param  n: int
        文章中の文の数
    @param  e: float
        許容誤差ε
    @return: list
        LexRank
    """
    transposed_matrix = cosine_matrix.T
    sentences_count = n

    p_vector = numpy.array([1.0 / sentences_count] * sentences_count)

    lambda_val = 1.0

    while lambda_val > e:
        next_p = numpy.dot(transposed_matrix, p_vector)
        lambda_val = numpy.linalg.norm(numpy.subtract(next_p, p_vector))
        p_vector = next_p

    return p_vector

## 試す

In [28]:
import glob
text = ''
files = glob.glob('./data/*.txt')
for file in files:
    with open(file, 'r') as f:
        text += f.read()
documents = text.split("\n")

In [29]:
# 分かち書き
import MeCab
tagger = MeCab.Tagger('-Owakati')
def wakati(tagger, document):
     return tagger.parse(document).split()

In [40]:
sentences = list(map(lambda doc: wakati(tagger, doc), documents))
sentences = list(filter(lambda s: len(s) > 0, sentences))

In [39]:
result = lex_rank(sentences, len(sentences), 0.1)
scores = []
for (sentence, rating) in result:
    scores.append({"score": rating, "sentence": ''.join(sentence)})

In [42]:
# 全文のスコアができたので、そのレートを高い順に出すdict化してレートソート？
sorted_score = sorted(list(map(lambda s: s["score"], scores)), reverse = True)
limit = sorted_score[2]
for s in scores:
    if(s["score"] < limit):
        continue
    print(s["sentence"])

そのアルゴリズムは、第一にそのデータが生成した潜在的機構の特徴を捉え、複雑な関係を識別（すなわち定量化）する。第二にその識別したパターンを用いて、新たなデータについて予測を行う。データは、観測された変数群のとりうる関係の具体例と見ることができる。一方、アルゴリズムは、機械学習者として観測されたデータの部分（訓練例などと呼ぶ）を学習することで、データに潜在する確率分布の特徴を捉え、学習によって得た知識を用いて、新たな入力データについて知的な決定を行う。
機械学習システムによっては、人間の直観によるデータ解析の必要性を排除しようとしているが、人間と機械の協調的相互作用を取り入れたものもある。しかし、そもそもシステムのデータ表現方法やデータの特徴を探る機構は、人間が設計したものであり、人間の直観を完全に排除することはできない。
という例外はあるが、基本的に学会も学術誌も別々である。それらの間の混同の最大の原因は、それらの基本的前提に由来する。機械学習では、既知の知識を再生成できるかどうかで性能を評価するが、データマイニングではそれまで「未知」だった知識を発見することが重視される。したがって、既知の知識によって評価するなら「教師なしの技法」よりも「教師ありの技法」の方が容易に優れた結果を示すことができる。しかし、典型的なデータマイニングでは、訓練データが用意できないので、「教師ありの技法」を採用することができない。
